In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## עיבוד מבני נתונים (Reshaping DataFrames)

כאשר אנו מנתחים נתונים ניסיוניים, לעיתים יש צורך **לשנות את מבנה הטבלה** כך שתתאים לצורת הניתוח הרצויה.  
ב־Pandas קיימים כלים רבי־עוצמה לשינוי מבנה נתונים — כגון `pivot`, `melt`, ו־`stack`/`unstack`.

שינוי מבנה נתונים שימושי במיוחד במקרים בהם יש צורך:
- לאגד מדידות לפי ניסוי, תאריך או קטגוריה (wide → long).  
- ליצור טבלאות סיכום לפי חודשים או קבוצות (long → wide).  
- לחשב ערכים מצטברים (כמו סכום אנרגיה או ממוצע טמפרטורה) לפי פרמטרים שונים.

#### Long vs. Wide Format

במבנה **Long** כל שורה מייצגת תצפית אחת, למשל מדידה אחת של מתח וזרם בזמן נתון.  
במבנה **Wide**, הערכים של כל משתנה נפרדים לעמודות — לדוגמה, מתח וזרם של כל חודש בעמודות שונות.

לדוגמה:
- **Long:** עמודות `timestamp`, `category`, `net_power`
- **Wide:** עמודות `month`, `category_A`, `category_B`, ...

המעבר בין הצורות נעשה בעזרת `pivot` (או `pivot_table`) ו־`melt`.

#### Pivot Table עם פונקציות אגרגציה

הפונקציה `pivot_table()` מאפשרת ליצור טבלת סיכום (aggregation) לפי שני ממדים או יותר — לדוגמה, סכום הספק נטו (`net_power`) לפי חודש וקטגוריית ניסוי.  
ניתן להשתמש בפרמטר `aggfunc` כדי להגדיר כיצד לאחד ערכים כפולים (למשל `sum`, `mean`, `max`).

#### Melt — החזרה לפורמט Long

הפונקציה `melt()` מבצעת את הפעולה ההפוכה ל־`pivot`, ומחזירה את הנתונים למבנה ארוך.  
זוהי דרך נוחה לבדוק שקבצי הנתונים שנוצרו בפעולות pivot/pivot_table זהים למקור מבחינת תוכן, גם אם שונים במבנה.

### תרגילים: שינוי מבנה נתונים ניסיוניים

בתרגילים הבאים נשתמש בנתונים ניסיוניים הכוללים מדידות הספק נטו (`net_power`) שנאספו מניסויים שונים לאורך השנה.  
ניצור טבלת סיכום לפי חודש וקטגוריה, ולאחר מכן נחזיר אותה למבנה הארוך כדי לוודא שהמידע נשמר במדויק.

#### תרגיל 1: יצירת Pivot Table לפי חודש וקטגוריה

בתרגיל זה נבנה טבלת סיכום של **הספק נטו (net_power)** לפי חודש וקטגוריית ניסוי.  
נניח שהניסוי בוצע במעבדה במספר סבבים, וכל סבב שייך לקטגוריה אחרת (A, B, C).  
נחשב את סכום ההספק הכולל בכל חודש ובכל קטגוריה — פעולה שמאפשרת לזהות באילו חודשים התקבלו ערכי הספק גבוהים במיוחד.


In [20]:
# --- Exercise 1: Build pivot_table by month and category ---
import pandas as pd

# Create sample experimental data (measurements of electrical power)
data = {
    "timestamp": pd.date_range("2025-01-01", periods=12, freq="10D"),
    "category": ["A", "A", "B", "B", "A", "C", "B", "C", "A", "C", "B", "A"],
    "net_power": [1.2, 0.8, 1.5, 1.1, 2.0, 2.3, 1.8, 2.6, 1.4, 2.9, 1.7, 2.2]  # in milliwatts
}

df = pd.DataFrame(data)

# Extract month abbreviation for grouping
df["month"] = df["timestamp"].dt.strftime("%b")

# Build pivot table to summarize total power per month and category
pivot_df = df.pivot_table(
    index="month",
    columns="category",
    values="net_power",
    aggfunc="sum"
)

print("Pivot Table: sum of net_power by month and category")
display(pivot_df)

Pivot Table: sum of net_power by month and category


category,A,B,C
month,,,
Apr,2.2,1.7,2.9
Feb,2.0,NaN,2.3
Jan,2.0,2.6,NaN
Mar,1.4,1.8,2.6


#### תרגיל 2: ניתוח והשוואה בין חודשים

לאחר יצירת טבלת הסיכום, נוכל לבצע ניתוח קצר:  
- באיזה חודש התקבלה **האנרגיה הכוללת הגבוהה ביותר** בכל קטגוריה?  
- האם קיימים חודשים שבהם לא נמדדו כלל נתונים בקטגוריה מסוימת (NaN)?  
- מה ההבדל בין מבנה ה־Pivot לבין מבנה הנתונים המקורי (Long)?

הערה: ניתן להיעזר בפונקציות `idxmax()` ו־`isna()` כדי לענות על שאלות אלו.


In [21]:
# --- Exercise 2: Analyze the pivoted data ---
# Find the month with the maximum total power for each category
max_months = pivot_df.idxmax()
print("Month with maximum total power per category:")
display(max_months)

# Identify missing values (months without data for a given category)
missing_data = pivot_df.isna()
print("Missing data (True = no measurements):")
display(missing_data)


Month with maximum total power per category:


category
A    Apr
B    Jan
C    Apr
dtype: object

Missing data (True = no measurements):


category,A,B,C
month,,,
Apr,False,False,False
Feb,False,True,False
Jan,False,False,True
Mar,False,False,False


#### תרגיל 3: החזרת הנתונים לפורמט ארוך (Long Format)

בשלב זה נחזיר את הנתונים מהטבלה הרחבה (Wide) למבנה ארוך (Long)  
באמצעות הפונקציה `melt()`.  
המבנה הארוך מתאים יותר לניתוח סטטיסטי, גרפים והשוואות בין קטגוריות שונות.


In [22]:
# --- Exercise 3: Convert pivoted data back to long format using melt ---
# Reset index to prepare for melting
pivot_reset = pivot_df.reset_index()

# Melt to long format
melted_df = pivot_reset.melt(
    id_vars="month",
    var_name="category",
    value_name="net_power_sum"
)

print("Melted DataFrame (Long Format):")
display(melted_df)


Melted DataFrame (Long Format):


,month,category,net_power_sum
0,Apr,A,2.2
1,Feb,A,2.0
2,Jan,A,2.0
3,Mar,A,1.4
4,Apr,B,1.7
5,Feb,B,NaN
6,Jan,B,2.6
7,Mar,B,1.8
8,Apr,C,2.9
9,Feb,C,2.3


#### תרגיל 4: בדיקת שימור הנתונים לאחר שינוי מבנה

נבדוק האם סך כל ערכי ההספק במבנה הארוך זהה לסכום המקורי לפני פעולת ה־pivot.  
בדיקה זו חשובה כדי לוודא ששום מידע לא אבד בתהליך השינוי בין Long ↔ Wide.

In [23]:
# --- Exercise 4: Verify data equivalence between long and pivot forms ---
original_sum = df["net_power"].sum()
melted_sum = melted_df["net_power_sum"].sum()

print(f"Original total power: {original_sum:.2f} mW")
print(f"After melt total power: {melted_sum:.2f} mW")

if abs(original_sum - melted_sum) < 1e-6:
    print("Data integrity verified — no loss during reshaping.")
else:
    print("Data mismatch detected — check aggregation parameters.")


Original total power: 21.50 mW
After melt total power: 21.50 mW
Data integrity verified — no loss during reshaping.


`````{admonition} סיכום
:class: tip
בתרגילים אלו:
- יצרנו טבלת סיכום (`pivot_table`) של הספקים לפי חודש וקטגוריה ניסויית.  
- ניתחנו מגמות והשוואות בין חודשים שונים.  
- החזרנו את הנתונים לפורמט Long באמצעות `melt()`.  
- בדקנו ששמירת הנתונים עקבית ואמינה גם לאחר שינוי המבנה.

טכניקות אלו חיוניות בניתוח נתונים פיזיקליים רב־ממדיים — כגון ניסויים חוזרים בתנאים שונים,  
חישוב ממוצעים חודשיים של אנרגיה, או השוואת התנהגות חומרים בין טמפרטורות שונות.
`````

In [24]:
import json
from jupyterquiz import display_quiz

quiz_json = \
'''
[{
  "question": "בעת ניתוח נתוני ניסוי במעבדה, ברצונך לחשב את סכום ההספק הכולל (net_power) בכל חודש ובכל קטגוריה, ולאחר מכן להחזיר את הנתונים לפורמט ארוך לצורך ניתוח נוסף.<br><br>איזה רצף פעולות בפנדס מתאים לכך?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "שימוש ב־pivot_table עם aggfunc='sum' כדי ליצור טבלת סיכום, ולאחר מכן המרה עם melt כדי לחזור לפורמט ארוך.",
      "correct": true,
      "feedback": "נכון מאוד! pivot_table מאפשר לאחד נתונים לפי מפתחות (כמו חודש וקטגוריה), ו־melt מחזיר את הנתונים למבנה ארוך המתאים להשוואה וניתוח נוסף."
    },
    {
      "answer": "שימוש ב־stack כדי להפוך עמודות לשורות, ואז ב־groupby כדי לסכם לפי חודש וקטגוריה.",
      "correct": false,
      "feedback": "לא מדויק — stack מתאים למבני ריבוי רמות (MultiIndex), אך אינו מחליף את pivot_table באגרגציה ישירה."
    },
    {
      "answer": "שימוש ב־transpose כדי לסובב את כל הנתונים ואז ב־merge כדי לאחד בין חודשים.",
      "correct": false,
      "feedback": "לא נכון — transpose משנה את כל הטבלה מבלי להתחשב במפתחות או בערכים, ו־merge אינו רלוונטי כאן."
    },
    {
      "answer": "שימוש ב־unstack ולאחר מכן ב־pivot כדי ליצור סיכום חודשי.",
      "correct": false,
      "feedback": "לא — unstack הופך רמות אינדקס לעמודות, אך אינו מבצע חישוב אגרגציה כמו pivot_table."
    }
  ]
}]
'''

myquiz = json.loads(quiz_json)
display_quiz(myquiz)


<IPython.core.display.Javascript object>